# Настройки

## Импортируем библиотеки

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

## Устанавливаем связь с базой данных

In [2]:
# устанавливаем параметры соединения
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con).head(1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


# Изучаем структуру таблиц базы данных

## `books`

In [3]:
query = '''SELECT * FROM books LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


## `authors`

In [4]:
query = '''SELECT * FROM authors LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt


## `publishers`

In [5]:
query = '''SELECT * FROM publishers LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con).head(1)

,publisher_id,publisher
0,1,Ace


## `ratings`

In [6]:
query = '''SELECT * FROM ratings LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


## `reviews`

In [7]:
query = '''SELECT * FROM reviews LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


# Задания

## Считаем сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-01-01'

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


### Вывод
- После 1 января 2000 года вышло 

**819 книг.**

## Считаем количество обзоров и среднюю оценку для каждой книги .

In [15]:
query = '''
SELECT
book_id,
COUNT (book_id) AS reviews_num
FROM reviews
GROUP BY book_id
ORDER BY reviews_num DESC
LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,reviews_num
0,948,7
1,750,6
2,695,6
3,963,6
4,696,6


In [10]:
query = '''
WITH books_rt AS (   
        SELECT b.book_id,
        b.title,
        AVG(rt.rating) as avg_rating
        FROM books as b
        LEFT JOIN ratings as rt ON b.book_id=rt.book_id
        GROUP BY 1, 2)


SELECT 
br.book_id,
br.title,
br.avg_rating,
COUNT(rv.review_id) AS num_of_reviews
FROM books_rt AS br
LEFT JOIN reviews as rv ON br.book_id=rv.book_id
GROUP BY 1, 2, 3
ORDER BY avg_rating DESC

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,avg_rating,num_of_reviews
0,938,Triptych (Will Trent #1),5.00,2
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,0
2,390,Light in August,5.00,2
3,441,Misty of Chincoteague (Misty #1),5.00,2
4,901,The Walking Dead Book One (The Walking Dead #...,5.00,2
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3
996,371,Junky,2.00,2
997,202,Drowning Ruth,2.00,3
998,316,His Excellency: George Washington,2.00,2


### Вывод 
- Получена информация по 1000 книг
- Минимальный рейтинг 1,5, максимальный 5
- У 6 книг ни одного обзора
- Лидер имеет 7 обзоров 

## Определяем издательство, которое выпустило наибольшее число "толстых" книг (количество страниц больше 50)

In [11]:
query = '''SELECT publisher,
COUNT(book_id) AS books_num
FROM publishers AS p
LEFT JOIN books AS b ON b.publisher_id=p.publisher_id
WHERE num_pages >50
GROUP BY 1
ORDER BY books_num DESC
LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_num
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


### Вывод
Больше всего книг (42 наименования) толще 50 страниц выпустило издательство 

**Penguin Books.**

## Определяем автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)

In [12]:
query = '''WITH books_list AS (
    SELECT book_id,    
    COUNT(rating_id) ratings_num,
    AVG (rating) rating
    FROM ratings
    GROUP BY 1
    HAVING COUNT(rating_id)>=50)

SELECT b.author_id,
a.author,
AVG(bl.ratings_num) avg_rt_num,
AVG(bl.rating) AS average_rating
FROM books_list AS bl
LEFT JOIN books AS b ON b.book_id=bl.book_id
LEFT JOIN authors AS a ON b.author_id=a.author_id
GROUP BY 1, 2 
ORDER BY average_rating DESC
LIMIT 15

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rt_num,average_rating
0,236,J.K. Rowling/Mary GrandPré,77.5,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,53.0,4.264151
2,240,J.R.R. Tolkien,81.0,4.258446
3,376,Louisa May Alcott,52.0,4.192308
4,498,Rick Riordan,62.0,4.080645
5,621,William Golding,71.0,3.901408
6,235,J.D. Salinger,86.0,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,57.0,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,66.0,3.787879
9,106,Dan Brown,71.5,3.754540


### Вывод 
Автор с самой высокой средней оценкой книг, которые были оценены 50 или более раз, это писательница с необычными ФИО:

**J.K. Rowling/Mary GrandPré.**

## Считаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [16]:
query = ''' 

WITH reviewer_list AS 
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 48)

SELECT AVG(rc.reviews_num) avg_review_num_by_user
FROM (SELECT r.username,
COUNT (r.review_id) reviews_num
FROM reviews AS r
RIGHT JOIN reviewer_list AS rl ON rl.username=r.username
GROUP BY r.username) AS rc


'''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review_num_by_user
0,24.0


### Вывод
Среднее количество *обзоров* от пользователей, которые поставили более 48 *оценок*:

**24 обзора**

# Выводы
По результатам исследования можно сделать следующие выводы:
- Самый популярный автор в базе данных J.K. Rowling/Mary GrandPré
- Самое производительное издательство (по книгам толще 50 страниц) - Penguin Books.
- Среднее количество обзоров от активных пользователей сервиса (т.е. таких, которые поставили более 48 оценок книгам) 24.